In [1]:
# 1

# New Notebook to run the individual_flashes_plots.py routine
# that will be used to test the new LAMPFLASH files from
# 15 July 2021, which requires taking fresh RAWTAG files
# and running CalCOS on them with WAVECOR set to PERFORM
# so that they create LAMPFLASH files that can be compared
# with LAMPFLASH files that have *not* been shifted
# (which already exist).
#
# I re-downloaded the fresh files from /grp/hst/cos2/cosmo/16469/
# and put them at
# /Users/ahirschauer/Documents/Year4/04-2021/Lamp_Tabs/ahirscha58916/data_backup_22July2021
# where the .fits files and this .ipynb file were placed in
# a further sub-directory /fits_files/.
#
# This should be a quick Notebook to run, since all I want to do
# is run CalCOS on the data, but I need to make sure that all of
# the header parameters are set properly, which is to say
# more or less copied from the initial LAMPTAB creation Notebook.


In [2]:
# 2

# The first thing we need to do is import the relevant packages.

import calcos
from astropy.io import fits
import glob
import numpy as np

# Obviously we'll need CalCOS to process the COS data, but also
# astropy's fits to handle the .fits files, glob (which allows for
# manipulating data with similar file names), and numpy to do some
# sorting of the data based on .fits file header information.


In [3]:
# 3

# In my directory with all my data, I want to specify only some files
# to work on, from the vast assortment of files (I downloaded both the
# raw files and the processed files; clearly all we really need here are
# the unprocessed, raw files).
#
# Set a variable called "rawtags" with glob to specify only the files in
# this working directory that have the word "rawtag" in it:

rawtags = glob.glob('*_rawtag_*.fits')


In [4]:
# 4

# From here, we can print out a list of the files set by the command above,

#print(rawtags)

# or print out the whole header of the first file (index[0]),

#fits.getheader(rawtags[0])

# but the next *useful* thing to do is set a consistent 'RANDSEED' value
# for every rawtag file, which will be used for important things later on.

for myfile in rawtags:
    fits.setval(myfile, 'RANDSEED', value=123456789)
    
# This dinky for loop takes all the files defined by "rawtags", then uses
# the astropy fits package to set a value for 'RANDSEED' to the very creative
# value of 123456789.
#
# (Note: "myfile" in this case specifies the very last file set by "rawtags".)


In [5]:
# 5

# We can print out the 'RANDSEED' value to show that the last command did its job,

#fits.getval(myfile, 'RANDSEED')

# but that's only if you really need to prove it to yourself.

# What we need to do now, as stipulated in the ISRs for similar work done, is to
# set the various fits header parameters to what they need to be for when we run
# them through CalCOS, in order for them to run the way we want them to.

for myfile in rawtags:
    if fits.getval(myfile, 'OPT_ELEM') == 'G140L':
        fits.setval(myfile, 'FLATCORR', value='PERFORM')
        fits.setval(myfile, 'WAVECORR', value='PERFORM')
        fits.setval(myfile, 'TRCECORR', value='OMIT')
        fits.setval(myfile, 'ALGNCORR', value='OMIT')
        fits.setval(myfile, 'XTRCTALG', value='BOXCAR')
        fits.setval(myfile, 'X1DCORR', value='PERFORM')
        fits.setval(myfile, 'BACKCORR', value='OMIT')
        fits.setval(myfile, 'FLUXCORR', value='OMIT')
        fits.setval(myfile, 'HELCORR', value='OMIT')
        fits.setval(myfile, 'GEOCORR', value='PERFORM')
        fits.setval(myfile, 'YWLKCORR', value='PERFORM')
        fits.setval(myfile, 'TEMPCORR', value='PERFORM')
        fits.setval(myfile, 'IGEOCORR', value='PERFORM')
        fits.setval(myfile, 'LAMPTAB', value='/grp/hst/cos2/LP5_ERA/files_to_use/LP3_G140L_fuv_15July2021_lamp.fits')
        fits.setval(myfile, 'DISPTAB', value='/grp/hst/cos2/LP5_ERA/files_to_use/LP3_G140L_fuv_15July2021_disp.fits')
        fits.setval(myfile, 'XTRACTAB', value='/grp/hst/cos2/LP5_ERA/files_to_use/lp3_1dx_070721.fits')
        fits.setval(myfile, "LIFE_ADJ", value=3)
    elif fits.getval(myfile, 'OPT_ELEM') == 'G130M':
        fits.setval(myfile, 'FLATCORR', value='PERFORM')
        fits.setval(myfile, 'WAVECORR', value='PERFORM')
        fits.setval(myfile, 'TRCECORR', value='OMIT')
        fits.setval(myfile, 'ALGNCORR', value='OMIT')
        fits.setval(myfile, 'XTRCTALG', value='BOXCAR')
        fits.setval(myfile, 'X1DCORR', value='PERFORM')
        fits.setval(myfile, 'BACKCORR', value='OMIT')
        fits.setval(myfile, 'FLUXCORR', value='OMIT')
        fits.setval(myfile, 'HELCORR', value='OMIT')
        fits.setval(myfile, 'GEOCORR', value='PERFORM')
        fits.setval(myfile, 'YWLKCORR', value='PERFORM')
        fits.setval(myfile, 'TEMPCORR', value='PERFORM')
        fits.setval(myfile, 'IGEOCORR', value='PERFORM')
        fits.setval(myfile, 'LAMPTAB', value='/grp/hst/cos2/LP5_ERA/files_to_use/LP5_G130M_fuv_15July2021_lamp.fits')
        fits.setval(myfile, 'DISPTAB', value='/grp/hst/cos2/LP5_ERA/files_to_use/LP5_G130M_fuv_15July2021_disp.fits')
        fits.setval(myfile, 'XTRACTAB', value='/grp/hst/cos2/LP5_ERA/files_to_use/lp5_1dx_070721.fits')
        fits.setval(myfile, 'FLATFILE', value='/grp/hst/cos2/LP5_ERA/files_to_use/g130m_xiter2_flat.fits')
        fits.setval(myfile, "LIFE_ADJ", value=5)
        
# Confirm that these parameters are set properly before proceeding!
#
# We are running this all first for the G140L/800 data.
# ^ UPDATE: But now also for the G130M data.
#
# NB the multiple XTRACTAB lines, which got commented out and replaced once
# new, updated versions of these files became available.


In [6]:
# 6

# Next we need to specify the special lamp flash parameters to match what the
# observations are doing in the APT file, whereby the lamp is on for 30 seconds,
# then off for a while (how long and how many times depends on the exposure),
# so that it builds up signal without getting too hot, which could damage it.
#
# For the science exposures of G130M, the exposure times are 210 seconds, with
# 120 seconds of total lamp on time (four flashes), while for the science exposures
# of G140L, the exposure times are 450 seconds, with 240 seconds of total lamp
# time (eight flashes).
#
# Also note that, for the beginning of each Visit, there's a single long exposure
# to settle the OSM, which is 1440 seconds for the two G130M Visits, but 1800 seconds
# for the single G140L Visit.
#
# We'll set this with a large "if" statement written be Elaine, which sets parameters
# like lamp duration (LMPDUR1), the start time of a flash (LMP_ON1), the end time of
# a flash (LMPOFF1), and the median time of that flash (LMPMED1).
#
# For each kind of exposure, we define this for two flashes, then note the number of
# flashes (NUMFLASH), and how to continue the pattern (set TAGFLASH to 'UNIFORMLY SPACED').
#
# Finally, note at what extension each of these parameters should be set to.

for myfile in rawtags:
    if fits.getval(myfile, 'OPT_ELEM') == 'G130M' and fits.getval(myfile, 'EXPTIME', ext=1) < 220.:
        LMPDUR1 = 30.0
        LMP_ON1 = 0.0
        LMPOFF1 = 30.0
        LMPMED1 = 15.00000
        LMPDUR2 = 30.0
        LMP_ON2 = 60.0
        LMPOFF2 = 90.0
        LMPMED2 = 75.00000
        fits.setval(myfile, 'LMPDUR1', value=LMPDUR1, ext=1)
        fits.setval(myfile, 'LMP_ON1', value=LMP_ON1, ext=1)
        fits.setval(myfile, 'LMPOFF1', value=LMPOFF1, ext=1)
        fits.setval(myfile, 'LMPMED1', value=LMPMED1, ext=1)
        fits.setval(myfile, 'LMPDUR2', value=LMPDUR2, ext=1)
        fits.setval(myfile, 'LMP_ON2', value=LMP_ON2, ext=1)
        fits.setval(myfile, 'LMPOFF2', value=LMPOFF2, ext=1)
        fits.setval(myfile, 'LMPMED2', value=LMPMED2, ext=1)
        fits.setval(myfile, 'NUMFLASH', value=4, ext=1)
        fits.setval(myfile, 'TAGFLASH', value='UNIFORMLY SPACED', ext=0)
    elif fits.getval(myfile, 'OPT_ELEM') == 'G130M' and fits.getval(myfile, 'EXPTIME', ext=1) > 220.:
        LMPDUR1 = 30.0
        LMP_ON1 = 0.0
        LMPOFF1 = 30.0
        LMPMED1 = 15.00000
        LMPDUR2 = 30.0
        LMP_ON2 = 120.0
        LMPOFF2 = 150.0
        LMPMED2 = 135.00000
        fits.setval(myfile, 'LMPDUR1', value=LMPDUR1, ext=1)
        fits.setval(myfile, 'LMP_ON1', value=LMP_ON1, ext=1)
        fits.setval(myfile, 'LMPOFF1', value=LMPOFF1, ext=1)
        fits.setval(myfile, 'LMPMED1', value=LMPMED1, ext=1)
        fits.setval(myfile, 'LMPDUR2', value=LMPDUR2, ext=1)
        fits.setval(myfile, 'LMP_ON2', value=LMP_ON2, ext=1)
        fits.setval(myfile, 'LMPOFF2', value=LMPOFF2, ext=1)
        fits.setval(myfile, 'LMPMED2', value=LMPMED2, ext=1)
        fits.setval(myfile, 'NUMFLASH', value=12, ext=1)
        fits.setval(myfile, 'TAGFLASH', value='UNIFORMLY SPACED', ext=0)
    elif fits.getval(myfile, 'OPT_ELEM') == 'G140L' and fits.getval(myfile, 'EXPTIME', ext=1) < 460.:
        LMPDUR1 = 30.0
        LMP_ON1 = 0.0
        LMPOFF1 = 30.0
        LMPMED1 = 15.00000
        LMPDUR2 = 30.0
        LMP_ON2 = 60.0
        LMPOFF2 = 90.0
        LMPMED2 = 75.00000
        fits.setval(myfile, 'LMPDUR1', value=LMPDUR1, ext=1)
        fits.setval(myfile, 'LMP_ON1', value=LMP_ON1, ext=1)
        fits.setval(myfile, 'LMPOFF1', value=LMPOFF1, ext=1)
        fits.setval(myfile, 'LMPMED1', value=LMPMED1, ext=1)
        fits.setval(myfile, 'LMPDUR2', value=LMPDUR2, ext=1)
        fits.setval(myfile, 'LMP_ON2', value=LMP_ON2, ext=1)
        fits.setval(myfile, 'LMPOFF2', value=LMPOFF2, ext=1)
        fits.setval(myfile, 'LMPMED2', value=LMPMED2, ext=1)
        fits.setval(myfile, 'NUMFLASH', value=8, ext=1)
        fits.setval(myfile, 'TAGFLASH', value='UNIFORMLY SPACED', ext=0)
    elif fits.getval(myfile, 'OPT_ELEM') == 'G140L' and fits.getval(myfile, 'EXPTIME', ext=1) > 460.:
        LMPDUR1 = 30.0
        LMP_ON1 = 0.0
        LMPOFF1 = 30.0
        LMPMED1 = 15.00000
        LMPDUR2 = 30.0
        LMP_ON2 = 120.0
        LMPOFF2 = 150.0
        LMPMED2 = 135.00000
        fits.setval(myfile, 'LMPDUR1', value=LMPDUR1, ext=1)
        fits.setval(myfile, 'LMP_ON1', value=LMP_ON1, ext=1)
        fits.setval(myfile, 'LMPOFF1', value=LMPOFF1, ext=1)
        fits.setval(myfile, 'LMPMED1', value=LMPMED1, ext=1)
        fits.setval(myfile, 'LMPDUR2', value=LMPDUR2, ext=1)
        fits.setval(myfile, 'LMP_ON2', value=LMP_ON2, ext=1)
        fits.setval(myfile, 'LMPOFF2', value=LMPOFF2, ext=1)
        fits.setval(myfile, 'LMPMED2', value=LMPMED2, ext=1)
        fits.setval(myfile, 'NUMFLASH', value=15, ext=1)
        fits.setval(myfile, 'TAGFLASH', value='UNIFORMLY SPACED', ext=0)

# It may not be the most elegant code, but it'll do the job just fine!


In [7]:
# 7

# The next step in the process is to actually run CalCOS on the data.
#
# First, to account for a weird idiosyncrasy whereby CalCOS automatically
# processes rawtag_b files after running on rawtag_a files, then crashes when
# it sees rawtag_b files that have already been processed, let's define
# a new grouping of files that are all the rawtag_a files alone:

rawtagsa = glob.glob('*_rawtag_a.fits')

# Now we can run CalCOS on our data, outputting to a new directory made
# to keep things clean (pick your own directory path):

#for myfile in rawtagsa:
#    calcos.calcos(myfile, outdir='/Users/ahirschauer/Documents/Year4/04-2021/Lamp_Tabs/ahirscha58916/output_15July2021/G140L/CalCOS_out_22July2021')

# Be aware that, if you're running this all locally, this step will take a decent
# amount of time (15-20 minutes, maybe).

# ^ Commented out the CalCOS running part of this Cell so that
#   if/when I re-run this Notebook, I'm not re-running
#   CalCOS again, since it takes forever (and isn't necessary).


In [8]:
# 8

# I'm going to copy+paste the whole routine right here as a test, with new directories:

from astropy.io import fits
import glob
import os
import numpy as np

from matplotlib import pyplot as plt

#-------------------------------------------------------------------------------

def get_shifts(shifts):

    new_shifts = []
    for shift in shifts[1:]:
        new_shifts.append(shift-shifts[0])

    #print('{:5} {:3} {:6} {:10}'.format(cenwave, fp, segment, max_shift ))
    return new_shifts

#-------------------------------------------------------------------------------

def make_figures(lamp_list):

    for lamp in lamp_list:

        outdir = '/Users/ahirschauer/Documents/Year4/07-2021/LAMPTABs/Individual_Flashes_Plots/output/'
        initial_dir = '/Users/ahirschauer/Documents/Year4/07-2021/LAMPTABs/Individual_Flashes_Plots/pre-shift/'
        new_dir = '/Users/ahirschauer/Documents/Year4/07-2021/LAMPTABs/Individual_Flashes_Plots/post-shift/'

        lamp_init = os.path.join(initial_dir, lamp)
        lamp_fin = os.path.join(new_dir, lamp)

        colors = {1: 'red',
                  2: 'blue',
                  3: 'orange',
                  4: 'green'}

        for segment in ['FUVA', 'FUVB']:
            plt.figure(figsize=(10, 5))
            shifts = []
            for lamp_file in [lamp_init, lamp_fin]:

                if 'old' in lamp_file:
                    marker = 'x'
                else:
                    marker = '.'

                with fits.open(lamp_file) as lf: # reading in the lamptab file
                    lamptab = os.path.join("/grp/hst/cos2/LP5_ERA/files_to_use/LP3_G140L_fuv_15July2021_lamp.fits"), lf[0].header['LAMPTAB'].split('$')[-1]

                    print ('-----------------')
                    print (lamptab)
                    cen = lf[0].header['cenwave']
                    fp = lf[0].header['fpoffset']
                    print (cen,fp+3,segment)
                    with fits.open(lamptab) as lt:
                        wh_tab = np.where((lt[1].data['cenwave'] == cen) &
                                          (lt[1].data['fpoffset'] == fp) &
                                          (lt[1].data['segment'] == segment))

                        if len(wh_tab[0]) == 0:
                            print('no {} {} {}'.format(cen, fp, segment))
                            continue
                        fp_pix_shift = lt[1].data['FP_PIXEL_SHIFT'][wh_tab] #finding FP_PIXSHIFT for that lampflash

                    wh_seg = np.where(lf[1].data['segment'] == segment)

                    #shift_disp is the shift between tagflash wavecal and template wavecal in dispersion direction
                    #i.e. the FP_shift + any drift
                    shift_disp = lf[1].data['SHIFT_DISP'][wh_seg]
                    print ('shift_disp:')
                    print (shift_disp)

                    shift_disp = shift_disp - fp_pix_shift
                    # since comparison was made between de-drifted corrtags and lamptemplate corrected for drift,
                    # the resultant value should be the FP shifts. Subtracting FP shifts should give zero
                    print ('shift_disp - FP_pixshift:')
                    print (shift_disp)

                    new_shifts = get_shifts(shift_disp)
                    print ('shifts relative to the first one (i.e. drift)')
                    print(new_shifts)

                    xvals=[2,3,4]
                    plt.plot(xvals,new_shifts, marker=marker, color=colors[fp+3], markersize=15, linestyle="None",
                             label='{} FP{}'.format(lamp_file.split('/')[-2], lf[0].header['FPPOS']))

                    shifts.append(new_shifts)

            plt.title('{} FP{} {}'.format(cen, fp+3, segment))
            plt.xlabel('Exposure after 1st')
            plt.ylabel('Drift Relative to 1st Exposure')
            plt.axhline(0, ls='--', color='black')
            plt.axhline(-0.5, ls=':', color='orchid')
            plt.axhline(0.5, ls=':', color='orchid')

            #plt.legend(bbox_to_anchor=(0.2, 0.98), ncol=4)
            plt.legend(loc='upper left', ncol=4)

            shifts = np.array(shifts)
            if len(shifts) == 0:
                plt.close()
                continue
            if np.max(abs(shifts)) > 1.0:
                plt.ylim(-2.0, 2.0)
#                plt.ylim(-1*(np.max(abs(shifts)))-0.1, (np.max(abs(shifts))+0.1) )
            else:
                plt.ylim(-1.0, 1.0)

            outfile = os.path.join(outdir, '{}_{}_{}_compare.png'.format(cen, fp+3, segment))
            plt.savefig(outfile)
            print('Saved: {}'.format(outfile))
            plt.close()

if __name__ == "__main__":

    outdir = '/Users/ahirschauer/Documents/Year4/07-2021/LAMPTABs/Individual_Flashes_Plots/output/'
    initial_dir = '/Users/ahirschauer/Documents/Year4/07-2021/LAMPTABs/Individual_Flashes_Plots/pre-shift/'
    new_dir = '/Users/ahirschauer/Documents/Year4/07-2021/LAMPTABs/Individual_Flashes_Plots/post-shift/'

    # grabbing the basenames of every lampflash in the newly calibrated directory
    lamp_bases = [os.path.basename(f) for f in glob.glob(os.path.join(new_dir, '*lampflash*'))]
    make_figures(lamp_bases)


-----------------
('/grp/hst/cos2/LP5_ERA/files_to_use/LP3_G140L_fuv_15July2021_lamp.fits', '15July2021_G140L_interim_lamp.fits')
800 3 FUVA


OSError: File-like object does not have a 'write' method, required for mode 'ostream'.

<Figure size 720x360 with 0 Axes>